In [1]:
import pandas as pd
import numpy as np
#logistics regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report

import matplotlib.pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn import linear_model
import scipy.stats as stat

class LogisticReg:
    """
    Wrapper Class for Logistic Regression which has the usual sklearn instance 
    in an attribute self.model, and pvalues, z scores and estimated 
    errors for each coefficient in 
    
    self.z_scores
    self.p_values
    self.sigma_estimates
    
    as well as the negative hessian of the log Likelihood (Fisher information)
    
    self.F_ij
    """
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        #### Get p-values for the fitted model ####
        denom = (2.0*(1.0+np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.array([np.sqrt(Cramer_Rao[i,i]) for i in range(Cramer_Rao.shape[0])]) # sigma for each coefficient
        z_scores = self.model.coef_[0]/sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values
        
        self.z_scores = z_scores
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.F_ij = F_ij

In [3]:
def summary(X,logit,logit1):
    names = list(X.columns)
    summary_90 = pd.DataFrame(index=names,columns=['Coefficient','p-value','<.10','<.05','<.01'])
    summary_90['Coefficient']=logit1.coef_[0]
    summary_90['p-value']=logit.p_values
    summary_90['Coefficient']=summary_90['Coefficient'].apply(lambda x: float('{:.2e}'.format(x)))
    summary_90['p-value']=summary_90['p-value'].apply(lambda x: float('{:.2e}'.format(x)))
    ten=[]
    five=[]
    one=[]
    for i in range(0,len(X.columns)):
        if summary_90['p-value'][i]<0.1:
            ten.append('*')
        else:
            ten.append('')
        if summary_90['p-value'][i]<0.05:
            five.append('*')
        else:
            five.append('')
        if summary_90['p-value'][i]<0.01:
            one.append('*')
        else:
            one.append('')
    summary_90['<.10']=ten
    summary_90['<.05']=five
    summary_90['<.01']=one
    intercept = float('{:.2e}'.format(logit1.intercept_[0]))
    inter=pd.DataFrame({'Coefficient': [intercept],'p-value':'','<.10':'','<.05':'','<.01':''},index=['Intercept'])
    summary_90 = pd.concat([inter,summary_90])
    return summary_90

In [4]:
data = pd.read_csv('UDPNY_gerardo.csv')

In [5]:
data['all_li_90'] = data['vli_90']+data['li_90']
data['all_li_00'] = data['vli_00']+data['li_00']
data['all_li_16'] = data['vli_16']+data['li_16']

In [6]:
l=[]
for i in range(0,len(data)):
    if ((data.Typology[i]=='LI - Ongoing Gentrification and/or Displacement') | 
        (data.Typology[i]=='LI - At Risk of Gentrification and/or Displacement')):
        l.append(1)
    else:
        l.append(0)

In [7]:
data['gen'] = l

In [8]:
data['TOD_pre_00'] = data['TOD_pre_00'].apply(lambda x: float(x))
data['TOD_00_10'] = data['TOD_00_10'].apply(lambda x: float(x))
data['TOD_10'] = data['TOD_10'].apply(lambda x: float(x))
data['TOD'] = data['TOD'].apply(lambda x: float(x))

In [9]:
logit_90 = LogisticReg(C = 10000)
logit1_90 = LogisticRegression(C = 10000)
logit_00 = LogisticReg(C = 10000)
logit1_00 = LogisticRegression(C = 10000)
logit_16 = LogisticReg(C = 10000)
logit1_16 = LogisticRegression(C = 10000)

In [10]:
data['hinc_90_norm'] = data['hinc_90']/np.max(data['hinc_90'])
data['hinc_00_norm'] = data['hinc_00']/np.max(data['hinc_00'])
data['hinc_16_norm'] = data['hinc_16']/np.max(data['hinc_16'])
data['empd15_norm'] = data['empd15']/np.max(data['empd15'])

In [11]:
X_90 = data[['hinc_90_norm', 'per_rent_90', 'per_nonwhite_90', 'TOD_pre_00', 'empd15_norm', 
                      'per_units_pre50']]
X_00 = data[['hinc_00_norm', 'per_rent_00', 'per_nonwhite_00', 'TOD_pre_00', 'TOD_00_10', 'empd15_norm', 
                      'per_units_pre50']]
X_16 = data[['hinc_16_norm', 'per_rent_16', 'per_nonwhite_16', 'TOD', 'empd15_norm', 
                      'per_units_pre50']]

In [12]:
X_90 = X_90.reset_index().drop('index',axis=1)
X_00 = X_00.reset_index().drop('index',axis=1)
X_16 = X_16.reset_index().drop('index',axis=1)

In [13]:
Y = data.gen 

In [14]:
Y = Y.reset_index().drop('index',axis=1)

In [15]:
logit_90.fit(X_90, Y.values.ravel())
logit1_90.fit(X_90, Y.values.ravel())
logit_00.fit(X_00, Y.values.ravel())
logit1_00.fit(X_00, Y.values.ravel())
logit_16.fit(X_16, Y.values.ravel())
logit1_16.fit(X_16, Y.values.ravel())

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
summary_90 = summary(X_90,logit_90,logit1_90)
summary_00 = summary(X_00,logit_00,logit1_00)
summary_16 = summary(X_16,logit_16,logit1_16)

In [17]:
summary_90

,Coefficient,p-value,<.10,<.05,<.01
Intercept,-2.870,,,,
hinc_90_norm,-6.500,3.36e-41,*,*,*
per_rent_90,0.797,2.28e-05,*,*,*
per_nonwhite_90,-0.214,0.317,,,
TOD_pre_00,2.160,1.51e-68,*,*,*
empd15_norm,-1.850,0.027,*,*,
per_units_pre50,1.640,1.38e-13,*,*,*


In [18]:
summary_00

,Coefficient,p-value,<.10,<.05,<.01
Intercept,-3.290,,,,
hinc_00_norm,-5.940,6.09e-34,*,*,*
per_rent_00,0.875,0.000137,*,*,*
per_nonwhite_00,0.375,0.0311,*,*,
TOD_pre_00,2.150,1.51e-73,*,*,*
TOD_00_10,0.405,0.338,,,
empd15_norm,-0.759,0.366,,,
per_units_pre50,1.650,2.07e-13,*,*,*


In [19]:
summary_16

,Coefficient,p-value,<.10,<.05,<.01
Intercept,-5.50000,,,,
hinc_16_norm,-0.00116,0.0306,*,*,
per_rent_16,2.60000,3e-33,*,*,*
per_nonwhite_16,0.62800,0.000318,*,*,*
TOD,1.94000,9.38e-66,*,*,*
empd15_norm,-2.36000,0.0043,*,*,*
per_units_pre50,1.65000,4.02e-18,*,*,*


In [20]:
pred_val_90 = logit1_90.predict(X_90)
pred_val_00 = logit1_00.predict(X_00)
pred_val_16 = logit1_16.predict(X_16)

In [21]:
print(accuracy_score(Y, pred_val_90),
accuracy_score(Y, pred_val_00),
accuracy_score(Y, pred_val_16))

0.8911463495209778 0.8921374297984803 0.8903204492897258
